## Vision Transformer (ViT)

In this assignment we're going to work with Vision Transformer. We will start to build our own vit model and train it on an image classification task.
The purpose of this homework is for you to get familar with ViT and get prepared for the final project.

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# VIT Implementation

The vision transformer can be seperated into three parts, we will implement each part and combine them in the end.

For the implementation, feel free to experiment different kinds of setup, as long as you use attention as the main computation unit and the ViT can be train to perform the image classification task present later.
You can read about the ViT implement from other libary: https://github.com/huggingface/pytorch-image-models/blob/main/timm/models/vision_transformer.py and https://github.com/pytorch/vision/blob/main/torchvision/models/vision_transformer.py

## PatchEmbedding
PatchEmbedding is responsible for dividing the input image into non-overlapping patches and projecting them into a specified embedding dimension. It uses a 2D convolution layer with a kernel size and stride equal to the patch size. The output is a sequence of linear embeddings for each patch.

In [3]:
class PatchEmbedding(nn.Module):
    def __init__(self, img_size=96, patch_size=16, num_hiddens=512):
        super().__init__()
        def _make_tuple(x):
            if not isinstance(x, (list, tuple)):
                return (x, x)
            return x
        img_size, patch_size = _make_tuple(img_size), _make_tuple(patch_size)
        self.num_patches = (img_size[0] // patch_size[0]) * (
            img_size[1] // patch_size[1])
        self.conv = nn.LazyConv2d(num_hiddens, kernel_size=patch_size,
                                  stride=patch_size)

    def forward(self, X):
        # Output shape: (batch size, no. of patches, no. of channels)
        return self.conv(X).flatten(2).transpose(1, 2)

## MultiHeadSelfAttention

This class implements the multi-head self-attention mechanism, which is a key component of the transformer architecture. It consists of multiple attention heads that independently compute scaled dot-product attention on the input embeddings. This allows the model to capture different aspects of the input at different positions. The attention outputs are concatenated and linearly transformed back to the original embedding size.

In [4]:
class DotProductAttention(nn.Module):
    """Scaled dot product attention."""
    def __init__(self, dropout):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

    # Shape of queries: (batch_size, no. of queries, d)
    # Shape of keys: (batch_size, no. of key-value pairs, d)
    # Shape of values: (batch_size, no. of key-value pairs, value dimension)
    # Shape of valid_lens: (batch_size,) or (batch_size, no. of queries)
    def forward(self, queries, keys, values, valid_lens=None):
        d = queries.shape[-1]
        # Swap the last two dimensions of keys with keys.transpose(1, 2)
        scores = torch.bmm(queries, keys.transpose(1, 2)) / math.sqrt(d)
        self.attention_weights =nn.functional.softmax(scores, dim=-1)
        return torch.bmm(self.dropout(self.attention_weights), values)

In [5]:

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__() # Call super().__init__() first
        self.num_heads = num_heads
        self.attention = DotProductAttention(dropout = 0.1)
        self.embed_dim = embed_dim # Store embed_dim
        self.head_dim = embed_dim // num_heads # Calculate and store head_dim here
        self.W_q = nn.LazyLinear(embed_dim, bias=False)
        self.W_k = nn.LazyLinear(embed_dim, bias=False)
        self.W_v = nn.LazyLinear(embed_dim, bias=False)
        self.W_o = nn.LazyLinear(embed_dim, bias=False)

    def forward(self, x):
        # Shape of queries, keys, or values:
        # (batch_size, no. of queries or key-value pairs, num_hiddens)
        # Shape of valid_lens: (batch_size,) or (batch_size, no. of queries)
        # After transposing, shape of output queries, keys, or values:
        # (batch_size * num_heads, no. of queries or key-value pairs,
        # num_hiddens / num_heads)
        # assert self.embed_dim % self.num_heads == 0
        queries = x
        keys = x
        values = x

        queries = self.transpose_qkv(self.W_q(queries))
        keys    = self.transpose_qkv(self.W_k(keys))
        values  = self.transpose_qkv(self.W_v(values))

        # Shape of output: (batch_size * num_heads, no. of queries,
        # num_hiddens / num_heads)
        # output = self.attention(queries, keys, values, valid_lens)
        d = queries.shape[-1]
        # Swap the last two dimensions of keys with keys.transpose(1, 2)
        scores = torch.bmm(queries, keys.transpose(1, 2)) / math.sqrt(d)
        attention_weights = torch.softmax(scores, dim=-1)
        output = torch.bmm(attention_weights, values)

        # Shape of output_concat: (batch_size, no. of queries, num_hiddens)
        output_concat = self.transpose_output(output)
        return self.W_o(output_concat)

    def transpose_qkv(self, X):
        # Transpose the query, key, or value tensors to split heads
        batch_size, seq_len, embed_dim = X.size()
        X = X.view(batch_size, seq_len, self.num_heads, self.head_dim)
        X = X.permute(0, 2, 1, 3)  # (batch_size, num_heads, seq_len, head_dim)
        return X.reshape(batch_size * self.num_heads, seq_len, self.head_dim)

    def transpose_output(self, X):
        # Transpose the output from multi-head attention to combine heads
        batch_size_times_heads, seq_len, head_dim = X.size()
        batch_size = batch_size_times_heads // self.num_heads
        X = X.view(batch_size, self.num_heads, seq_len, head_dim)
        X = X.permute(0, 2, 1, 3)  # (batch_size, seq_len, num_heads, head_dim)
        return X.contiguous().view(batch_size, seq_len, self.embed_dim)


In [6]:
num_hiddens, num_heads = 100, 5
attention = MultiHeadSelfAttention(num_hiddens, num_heads)
batch_size, num_queries, num_kvpairs = 2, 4, 6
valid_lens = torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))
attention(X).shape

torch.Size([2, 4, 100])

## TransformerBlock
This class represents a single transformer layer. It includes a multi-head self-attention sublayer followed by a position-wise feed-forward network (MLP). Each sublayer is surrounded by residual connections.
You may also want to use layer normalization or other type of normalization.

In [7]:
class ViTMLP(nn.Module):
    def __init__(self, mlp_num_hiddens, mlp_num_outputs, dropout=0.25):
        super().__init__()
        self.dense1 = nn.LazyLinear(mlp_num_hiddens)
        self.gelu = nn.GELU()
        self.dropout1 = nn.Dropout(dropout)
        self.dense2 = nn.LazyLinear(mlp_num_outputs)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x):
        #print(f"ViTMLP - Input shape: {x.shape}")
        x = self.dense1(x)
        #print(f"ViTMLP - After dense1: {x.shape}")
        x = self.gelu(x)
        #print(f"ViTMLP - After GELU: {x.shape}")
        x = self.dropout1(x)
        #print(f"ViTMLP - After dropout1: {x.shape}")
        x = self.dense2(x)
        #print(f"ViTMLP - After dense2: {x.shape}")
        x = self.dropout2(x)
        #print(f"ViTMLP - After dropout2: {x.shape}")
        return x


class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout):
        # A single Transformer block that includes multi-head self-attention and a feed-forward MLP layer.
        # Parameters:
        # ----------
        # embed_dim : int
        #     The dimensionality of the input embeddings. This represents the size of each input vector.
        # num_heads : int
        #     The number of attention heads. Multi-head attention allows the model to focus on different parts of the input sequence simultaneously.
        # mlp_dim : int
        #     The dimensionality of the hidden layer in the feed-forward MLP network. Controls the capacity of the MLP layer in the Transformer block.
        # dropout : float
        #     The dropout rate for regularization. Dropout is applied in the attention mechanism and the MLP to prevent overfitting.
        super().__init__()
        self.num_hiddens = embed_dim
        self.ln1 = nn.LayerNorm(embed_dim)
        self.attention = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ln2 = nn.LayerNorm(embed_dim)
        self.mlp = ViTMLP(mlp_dim, embed_dim, dropout)

    def forward(self, x):
        #print(f"TransformerBlock - Input shape: {x.shape}")
        # Apply layer normalization, followed by multi-head attention
        x_ln1 = self.ln1(x)
        #print(f"TransformerBlock - After ln1: {x_ln1.shape}")
        attn_output = self.attention(x_ln1)
        #print(f"TransformerBlock - After attention: {attn_output.shape}")
        x = x + attn_output
        #print(f"TransformerBlock - After adding attention output: {x.shape}")
        # Apply second layer normalization, followed by MLP
        x_ln2 = self.ln2(x)
        #print(f"TransformerBlock - After ln2: {x_ln2.shape}")
        mlp_output = self.mlp(x_ln2)
        #print(f"TransformerBlock - After MLP: {mlp_output.shape}")
        x = x + mlp_output
        #print(f"TransformerBlock - After adding MLP output: {x.shape}")
        return x



In [8]:
X = torch.ones((2, 100, 24))
encoder_blk = TransformerBlock(embed_dim=24, num_heads=8, mlp_dim=48, dropout=0.5)
encoder_blk.eval()
assert encoder_blk(X).shape == X.shape, "Output shape does not match input shape"

## VisionTransformer:
This is the main class that assembles the entire Vision Transformer architecture. It starts with the PatchEmbedding layer to create patch embeddings from the input image. A special class token is added to the sequence, and positional embeddings are added to both the patch and class tokens. The sequence of patch embeddings is then passed through multiple TransformerBlock layers. The final output is the logits for all classes

In [9]:
class VisionTransformer(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, embed_dim, num_heads, mlp_dim, num_layers, num_classes, dropout=0.1):
        super().__init__()
        #print("Initializing VisionTransformer")

        self.patch_embedding = PatchEmbedding(image_size, patch_size, embed_dim)
        #print(f"PatchEmbedding initialized with image_size={image_size}, patch_size={patch_size}, embed_dim={embed_dim}")

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        num_steps = self.patch_embedding.num_patches + 1  # Add the cls token
        self.pos_embedding = nn.Parameter(torch.randn(1, num_steps, embed_dim))
        #print(f"Position Embedding initialized with num_steps={num_steps}, embed_dim={embed_dim}")

        self.dropout = nn.Dropout(dropout)
        #print(f"Dropout layer initialized with dropout={dropout}")

        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module(f"block_{i}", TransformerBlock(embed_dim, num_heads, mlp_dim, dropout))
            #print(f"TransformerBlock {i} added to blks")

        self.head = nn.Sequential(nn.LayerNorm(embed_dim), nn.Linear(embed_dim, num_classes))
        #print("Head initialized with LayerNorm and Linear layer for classification")

    def forward(self, x):
        #print(f"Input shape: {x.shape}")

        x = self.patch_embedding(x)
        #print(f"After PatchEmbedding, shape: {x.shape}")

        cls_tokens = self.cls_token.expand(x.shape[0], -1, -1)
        #print(f"CLS token shape after expand: {cls_tokens.shape}")

        x = torch.cat((cls_tokens, x), dim=1)
        #print(f"Shape after concatenating CLS token: {x.shape}")

        x = x + self.pos_embedding
        #print("Added positional embeddings")

        x = self.dropout(x)
        #print("Applied dropout")

        x = self.blks(x)
        #print(f"After transformer blocks, shape: {x.shape}")

        output = self.head(x[:, 0])
        #print(f"Final output shape (after head): {output.shape}")

        return output


In [10]:
# Assuming `ViT` and `PatchEmbedding` classes are defined elsewhere and imported.
# Parameters for the ViT model
img_size = 32          # Example image size (32x32)
patch_size = 8         # Patch size (8x8)
num_hiddens = 64       # Number of hidden dimensions for the embeddings
mlp_num_hiddens = 128  # Number of hidden units in MLP
num_heads = 4          # Number of attention heads
num_blks = 2           # Number of Transformer blocks
emb_dropout = 0.20      # Dropout rate for embeddings
blk_dropout = 0.40      # Dropout rate in blocks
num_classes = 10       # Number of output classes

# Create a ViT model instance
vit_model = VisionTransformer(
    # img_size=img_size,
    # patch_size=patch_size,
    # num_hiddens=num_hiddens,
    # mlp_num_hiddens=mlp_num_hiddens,
    # num_heads=num_heads,
    # num_blks=num_blks,
    # emb_dropout=emb_dropout,
    # blk_dropout=blk_dropout,
    # num_classes=num_classes
    image_size = 32,
    patch_size = 8,
    in_channels = 3,
    embed_dim = 64,
    num_heads = 4,
    mlp_dim = 128,
    num_layers = 1,
    num_classes = 10,
    dropout=0.1
)

# Create a dummy input tensor (batch_size, channels, height, width)
batch_size = 4
input_tensor = torch.randn(batch_size, 3, img_size, img_size)

# Pass the input tensor through the model and print output shape
output = vit_model(input_tensor)
#print("Output shape:", output.shape)

# Sanity checks on the dimensions
assert output.shape == (batch_size, num_classes), f"Expected output shape {(batch_size, num_classes)}, but got {output.shape}"

# Additional Debugging Prints (Optional)
with torch.no_grad():
    # Check intermediate patch embedding shape
    patch_embed = vit_model.patch_embedding(input_tensor)
    print("Patch Embedding shape:", patch_embed.shape)
    expected_patch_embed_shape = (batch_size, (img_size // patch_size) ** 2, num_hiddens)
    assert patch_embed.shape == expected_patch_embed_shape, f"Expected patch embedding shape {expected_patch_embed_shape}, but got {patch_embed.shape}"

    # Check after adding class token
    cls_token_added = torch.cat((vit_model.cls_token.expand(batch_size, -1, -1), patch_embed), 1)
    print("Shape after adding class token:", cls_token_added.shape)
    expected_cls_token_shape = (batch_size, (img_size // patch_size) ** 2 + 1, num_hiddens)
    assert cls_token_added.shape == expected_cls_token_shape, f"Expected shape after adding class token {expected_cls_token_shape}, but got {cls_token_added.shape}"

    # Check after positional embeddings and dropout
    pos_embed_added = vit_model.dropout(cls_token_added + vit_model.pos_embedding)
    print("Shape after adding positional embeddings:", pos_embed_added.shape)
    assert pos_embed_added.shape == expected_cls_token_shape, f"Expected shape after adding positional embeddings {expected_cls_token_shape}, but got {pos_embed_added.shape}"

print("Sanity check passed successfully.")

Patch Embedding shape: torch.Size([4, 16, 64])
Shape after adding class token: torch.Size([4, 17, 64])
Shape after adding positional embeddings: torch.Size([4, 17, 64])
Sanity check passed successfully.


## Let's train the ViT!

We will train the vit to do the image classification with cifar100. Free free to change the optimizer and or add other tricks to improve the training

In [11]:
# Example usage:
image_size =  96 # TODO
patch_size =  16 # TODO
in_channels = 3# TODO
embed_dim =   512# TODO
num_heads =   8# TODO
mlp_dim =     2048# TODO
num_layers =  8  # TODO
num_classes = 100  # TODO
dropout =     0.2 # TODO
batch_size =  256 # TODO

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VisionTransformer(image_size, patch_size, in_channels, embed_dim, num_heads, mlp_dim, num_layers, num_classes, dropout).to(device)
input_tensor = torch.randn(1, in_channels, image_size, image_size).to(device)
#print(input_tensor.shape)
output = model(input_tensor)
#print(output.shape)

In [13]:
# Load the CIFAR-100 dataset
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.Resize(image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=8)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=8)

100%|██████████| 169M/169M [00:12<00:00, 13.2MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [14]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = # TODO


# Optimizer configuration
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=3e-6,
    weight_decay=0.05,
    betas=(0.9, 0.999)
)

# Learning rate scheduler
num_epochs = 200
warmup_epochs = 5
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=num_epochs - warmup_epochs
)

# training loop with warmup and gradient clipping

warmup_steps = warmup_epochs * len(trainloader)
step = 0

In [15]:
print(device)

cuda


In [16]:
# # Train the model
best_val_acc = 0.0

# Initialize scaler for mixed precision training
scaler = torch.amp.GradScaler('cuda')  # Updated from torch.cuda.amp.GradScaler()

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Warmup learning rate
        if step < warmup_steps:
            lr_scale = min(1., float(step + 1) / warmup_steps)
            for pg in optimizer.param_groups:
                pg['lr'] = lr_scale * 3e-4
        step += 1

        # Mixed precision training with updated autocast
        with torch.amp.autocast('cuda'):  # Updated from torch.cuda.amp.autocast()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        optimizer.zero_grad()
        scaler.scale(loss).backward()

        # Gradient clipping
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

        if i % 100 == 99:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(trainloader)}], '
                  f'Loss: {total_loss/100:.4f}')
            total_loss = 0

    # Update learning rate
    if epoch >= warmup_epochs:
        scheduler.step()

    # Validation
    model.eval()
    correct = 0
    total = 0
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            with torch.amp.autocast('cuda'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_acc = 100. * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}] Validation Accuracy: {val_acc:.2f}%, '
          f'Validation Loss: {val_loss/len(testloader):.4f}')

    # Save the best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
        }, 'best_model.pth')
        print(f'New best model saved with validation accuracy: {val_acc:.2f}%')

Epoch [1/200], Step [100/196], Loss: 4.5572
Epoch [1/200] Validation Accuracy: 7.60%, Validation Loss: 4.0859
New best model saved with validation accuracy: 7.60%
Epoch [2/200], Step [100/196], Loss: 3.9482
Epoch [2/200] Validation Accuracy: 16.37%, Validation Loss: 3.5202
New best model saved with validation accuracy: 16.37%
Epoch [3/200], Step [100/196], Loss: 3.4686
Epoch [3/200] Validation Accuracy: 21.56%, Validation Loss: 3.2292
New best model saved with validation accuracy: 21.56%
Epoch [4/200], Step [100/196], Loss: 3.1836
Epoch [4/200] Validation Accuracy: 25.32%, Validation Loss: 3.0333
New best model saved with validation accuracy: 25.32%
Epoch [5/200], Step [100/196], Loss: 3.0167
Epoch [5/200] Validation Accuracy: 27.98%, Validation Loss: 2.8840
New best model saved with validation accuracy: 27.98%
Epoch [6/200], Step [100/196], Loss: 2.8900
Epoch [6/200] Validation Accuracy: 29.09%, Validation Loss: 2.8177
New best model saved with validation accuracy: 29.09%
Epoch [7/200

KeyboardInterrupt: 

In [17]:
from google.colab import files

# Download the saved model file
files.download('best_model.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Please submit your best_model.pth with this notebook. And report the best test results you get.

##  The best model accurancy we get is 60.02%
